In [2]:
from langda import langda_solve
model = """
0.7::burglary.
langda(LLM:"earthquake! (100% sure)",FUP:"false").
langda(LLM:"The probablilty of alarm 1 sound an alarm is 0.9",FUP:"false").
0.8::p_alarm2.

langda(LLM:"alarm") :  burglary, earthquake, p_alarm1.
alarm - langda(LLM:"If there's earthquake but not burglary, alarm 2 will sound an alarm",FUP:"false").
query(alarm).
"""
model2 = """% Facts:
person(alice). person(bob).
langda(LLM:"Mary and Bob live close to each other and they very often meet in the morning.").
0.1::stress(X) :- person(X).
happy(alice) :-
    langda(LLM:"if alice is not in stress and meet bob, she's happy").

query(happy(alice))."""
model3 = """
langda(LLM:"What was the weather yesterday in Darmstadt? rainy or sunny?", LOT:"search").

langda(LLM:"According to the message from cleaner: '/* Cleaner */', choose to be true or false.").

% Causation: Analyzing cause and effect based on above all informations
langda(LLM:"Figure out if the ground is wet or not", FUP:"true").
query(ground_wet).
"""
model4 = """
button(red).button(green).button(blue).
choose(X) :-
    langda(LLM:"It says: /* Command */, what should I chooose?").
query(choose(X)).
"""
result = langda_solve('double_dc',model2,'deepseek-chat',langda_ext={"Cleaner":"I was fired last week."})

print(result)
from problog import get_evaluatable
special_result = get_evaluatable().create_from(result).evaluate()



### =========== processing init_node =========== ###
[{'LOT': None, 'NET': None, 'LLM': 'Mary and Bob live close to each other and they very often meet in the morning.', 'FUP': 'True', 'HEAD': '', 'HASH': 'A15BF20E'}, {'LOT': None, 'NET': None, 'LLM': "if alice is not in stress and meet bob, she's happy", 'FUP': 'True', 'HEAD': 'happy(alice) ', 'HASH': '20541B04'}]
{'60DAC5DA': '0.8::weather_yesterday(sunny); 0.2::weather_yesterday(rainy).', '63C6AC52': '0.9::cleaner_fired.', 'A912216E': 'ground_wet :- weather_yesterday(rainy).\nquery(ground_wet).'}
[{'A15BF20E': None}, {'20541B04': None}]
processing _decide_next_init ...
### =========== ### current round: 0 ### =========== ###
### =========== processing generate_node =========== ###
Executing first chain: Code generation with tools...


> Entering new AgentExecutor chain...
Here is the completed Problog code based on the requirements provided in the `<Langda>` blocks:

```problog
% Facts:
person(alice).
person(bob).
person(mary).

% M

KeyboardInterrupt: 

In [1]:
import json
from problog.program import PrologString
from problog import get_evaluatable
from problog.engine import DefaultEngine

# ---- 主模板（含“桩”） ----
template = """
1/3::prize(1); 1/3::prize(2); 1/3::prize(3).
select_door(1).
member(X,[X|T]).
member(X,[H|T]) :- member(X,T).
% 占位 :- dynamic langda_code/2.
open_door(A) :- langda_code("338175EC",Clause), call(Clause,A).
win_keep     :- langda_code("B73E5085",Clause), call(Clause).
win_switch :-
  member(A,[1,2,3]), \\+ select_door(A), prize(A), \\+ open_door(A).
query(win_keep).  query(win_switch).
"""

marco = """
:- module(langda_macro, [parse_langda/3]).
:- multifile user:term_expansion/2.

% -------- term_expansion 钩子 -------- %
user:term_expansion(langda(Opts), ExpandedCode) :-
        parse_langda(Opts, DSLClauses),
        ExpandedCode = DSLClauses.

"""

patches = [{
"C5B4A895": "0.5::open_door(A) ; 0.5::open_door(B) :- select_door(Selected), member(A, [1,2,3]), member(B, [1,2,3]), A \\= B, A \\= Selected, B \\= Selected, \\+ prize(A), \\+ prize(B)."
},
{
"338175EC": "select_door(Selected), member(A, [1,2,3]), A \\= Selected, \\+ prize(A), member(Other, [1,2,3]), Other \\= Selected, Other \\= A, prize(Other)."
},
{
"B73E5085": "select_door(Door), prize(Door)."
}]

# ---- 把模板编译成数据库对象 ----
engine = DefaultEngine()
db = engine.prepare(PrologString(template))

# ---- 逐条把补丁断言进数据库 ----
for mapping in patches:
    for key, code in mapping.items():
        for stmt in PrologString(code):
            db += stmt                  # on-the-fly 加入概率规则:contentReference[oaicite:2]{index=2}
        # db.assertz(f'langda_code("{key}", {stmt.head.name})')  # 建立索引
        db.add_all(f'langda_code("{key}", {stmt})')
# ---- 求解 ----
result = get_evaluatable().create_from(db).evaluate()  # :contentReference[oaicite:3]{index=3}
print(result)


AttributeError: 'str' object has no attribute 'functor'

In [7]:
from langda.utils.parser_v2 import integrated_code_parser

full_text_for_parse_test = '''
% =============================== % EXAMPLE % =============================== %
lann(mnist_net1:"decrib net1",[M]:"describe M",N,[0,1,"None"]:"desc someting") :: event1(M,N). % lann(what?)
lann(mnist_net2,[X],Y:"what is Y?",[2,3,"dog","horn"]:"output 2 and 3")    % (what_is_this(X,S):maybe?
:: funfunf(X,Y).     % have fun here)!

lann(mnist_net3,       % net work haha
[R],S:"what whatwhaitsat?",     % X and what Y: this Y! stop, lann(X,Y)
["this","thas","was","isss"]:"outoutoutout")    % (what_e q2wre g wads  cf :: event114(X,Y).
:: digit232(R,S).     % have fun here)!


% UAV properties
% initial_langda_content[i]ge ~ normal(90, 5).
% langda_content[i]ge_cost ~ normal(-0.1, 0.2).
% weight ~ normal(0.2, 0.1).

% Weather conditions
1/10::fog; 9/10::clear.

% Visual line of sight
vlos(X) :- 
    fog, distance(X, operator) < 50;
    clear, distance(X, operator) < 100;
    clear, over(X, bay), distance(X, operator) < 400.

langda(ans:"answer",LLM:"nothing"). % langda(LLM:"this is me").
% langda(X:"answer",LLM). % stop it
langda(why:"answer",NET:"[mlp(1,2,3,4,5,6)]").
langda(X:"answer",llm:"stop"). % why not here?
/*
% Sufficient langda_content[i]ge to return to operator
can_return(X) :- 
    B is initial_langda_content[i]ge, O is langda_content[i]ge_cost,
    D is distance(X, operator), 0 < B + (2 * O * D), langda(LLM:"how about this?")
*/ langda(NET:"[no(1,2,3)]",LLM:"This is crazy")        .       waht_coin(sdaw,A) :- zzz(X;Y),
langda(LLM,NET).

% Permits related to local features
permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).

valid_drone_flight(X) :- (langda(X:"Return",T,LLM:"there's a high building at: (100,200)",NET:"[mnist_net1(0,1), mnist_net2(2,3)]"), vlos(X));(can_return(X));langda(T:"anstimewer",NET).

coin_win() :-
    % the code need to be clean!
    langda(X:"return", T:"time", NET:"[mnist_net1(0,1), mnist_net2(2,3)]",              % a langda code for new stuff
    Y:"happy",              % why the code is happy?
    TOL:"[mnist_net1(0,1), mnist_net2(2,3)]",              % wewewewewewewe
    LLM:"If the code difference is 2, % it returns 1, otherwise it returns 0.")          % we use deepseek here!
    ,permits(X), langda(X:"return", Y:"fun", Z:"darmstadt", NET:"[position_net(there,here), audio(dog,child,horn)]",     % nothing useful % so quit!
    LLM:"If the we is 2,            % comment end
    and horn is there, we eat banana."). coin_this(X,Y,3) :- langda(X,LLM:"asdasda"),       % comment conisthis

    predicate(sd, X), asd(MMMA)
    . 

% Definition of a valid mission
landscape(X) :- 
    vlos(X), weight < 25, can_return(X); 
    permits(X), can_return(X).

'''

txt = '''
PH :: make_coin(Coin, PH).            % AD-fact (annotated disjunction)
coin(C) :- make_coin(C, 0.8).
% coin(C) :- fake_coin(C, 0.8).

/* -------------coins_r/3 — recursive accumulator--------------
 * +SC0 : running success counter
 * -SC  : total successes after CNT flips
 * +CNT : how many coins still need to be tossed
 * ------------------------------------------------------------ */
coins_r(SC, SC, 0).
coins_r(SC0, S, CNT) :-              % define coins_r/3
    CNT > 0,                         % Ensuring a Positive Counter
    langda(LLM:"
        With the counter CNT decreasing to 0, each step:
        • If the CNT-th coin turns up heads, the success
          count increases by 1;
        • Otherwise, it remains unchanged.
        Finally, all steps are accumulated to S."
    ).                               % Rules that will be generated

% /*----------------------- Public API --------------------------*/
total(S) :- coins_r(0, S, 4).        % Toss 4 coins and count the heads
query(total(_)).                     % ⬅ ProbLog query entry
'''
new, _, _, _ = integrated_code_parser(txt,'{{LANGDA}}')


LANGDA:NONE||CODE:PH :: make_coin(Coin, PH).|      COMMENT: 
LANGDA:NONE||CODE: |      COMMENT: % AD-fact (annotated disjunction)
LANGDA:NONE||CODE:coin(C) :- make_coin(C, 0.8).|      COMMENT: 
LANGDA:NONE||CODE:|      COMMENT: % coin(C) :- fake_coin(C, 0.8).
/* -------------coins_r/3 — recursive accumulator--------------
 * +SC0 : running success counter
 * -SC : total successes after CNT flips
 * +CNT : how many coins still need to be tossed
 * ------------------------------------------------------------ */
LANGDA:NONE||CODE:coins_r(SC, SC, 0).|      COMMENT: 
LANGDA:NONE||CODE:coins_r(SC0, S, CNT) :- |      COMMENT: % define coins_r/3
LANGDA:NONE||CODE: CNT > 0, |      COMMENT: % Ensuring a Positive Counter
LANGDA:NONE||CODE: |      COMMENT: 
LANGDA:BODY||CODE:langda(LLM:"|      COMMENT: 
LANGDA:BODY||CODE: With the counter CNT decreasing to 0, each step:|      COMMENT: 
LANGDA:BODY||CODE: • If the CNT-th coin turns up heads, the success|      COMMENT: 
LANGDA:BODY||CODE: count incr

In [ ]:
langdda = {"LOT":"","HASH":"ASDF","LLM":"asdasdasdw"}
if langdda.get("LOT"):
    print("a")

In [21]:
from langda import langda_solve

model = """
# 
langda(LLM:"the cleaner on plan yesterday was: Emily Johnson", LOT:"retrieve").

query(ground_cleaned(true)).
"""

result = langda_solve('double_dc',model,'deepseek-chat')

### =========== processing init_node =========== ###
{}
[{'F4DF5283': None}]
processing _decide_next_init ...
### =========== ### current round: 0 ### =========== ###
### =========== processing generate_node =========== ###
*** Generated New Code ***
 % Facts about the cleaner
cleaner("Emily Johnson").

% Query to check if the ground was cleaned
query(ground_cleaned(true)).

% Rule defining when the ground is considered cleaned
ground_cleaned(true) :- cleaner("Emily Johnson"), yesterday.

Executing second chain: Code formatting...
processing _decide_next_gnrt... #current round: 1
### =========== ### current round: 1 ### =========== ###
### =========== processing evaluate_node =========== ###
# 
ground_cleaned(true) :- cleaner("Emily Johnson"), yesterday.
query(ground_cleaned(true)).
Running problog_test_tool...
Error evaluating Problog model:
    toks = self.label_tokens(string, root_tokens)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zhenzhili/miniforge3/envs/lang

KeyboardInterrupt: 

In [18]:
import json
from problog.program import PrologString
from problog import get_evaluatable
from problog.engine import DefaultEngine
m = """% Your model here
% Your model here
% Earth constant
meter_per_degree(111194.93). % R * π / 180 ≈ 111194.93 meters/degree
% 主谓词：计算相对偏移
relative_offset(NorthOffset, EastOffset) :-
    Lat1 = 49.878091, Lon1 = 8.654052, Lat2 = 49.876294, Lon2 = 8.653355, 
    % 计算纬度差导致的南北偏移（米）
    NorthOffset is (Lat2 - Lat1) * 111194.93,
    
    % 计算参考点纬度的余弦值（需转换为弧度）
    Lat1_Rad is Lat1 * pi / 180,
    CosLat1 is cos(Lat1_Rad),
    
    % 计算经度差导致的东西偏移（米）
    EastOffset is (Lon2 - Lon1) * 111194.93 * CosLat1.

% 示例查询（使用您的坐标）
query(relative_offset(North, East))."""
result = get_evaluatable().create_from(m).evaluate()  # :contentReference[oaicite:3]{index=3}
print(result)

{relative_offset(-199.81728920958565,-49.94409296139008): 1.0}


In [21]:
import re

key = next(iter(result))   # 取出唯一 key
pattern = r"relative_offset\(\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)\s*\)"
match = re.search(pattern, str(key))

if match:
    x, y = match.groups()
    print(x, y)


-199.81728920958565 -49.94409296139008


In [88]:
danger_rules_string1 = f"""
% Your model here
% Earth constant
meter_per_degree(111194.93). % R * π / 180 ≈ 111194.93 meters/degree

langda(LLM:"According to the user: /* User */, the coordinate of the user is at:
please form as: user_location(X,Y).
(X and Y are longitude and latitude)", LOT:"search with 'The geographical coordinates of ... is'",FUP:"false").

langda(LLM:"According to the police: /* Police */, the coordinate of the special zone is at:
please form as: special_zone_location(X,Y).
(X and Y are longitude and latitude)",FUP:"false").

relative_offset(NorthOffset, EastOffset) :-
    langda(LLM:"The location of the danger zone relative to the user, in meters, calculated based on user_location and special_zone_location above",FUP:"false").

query(relative_offset(NorthOffset, EastOffset)).
query(user_location(North, East)).
"""
danger_rules_string2 = f"""
% Your model here
% Earth constant
meter_per_degree(111194.93). % R * π / 180 ≈ 111194.93 meters/degree

langda(LLM:"According to the user: /* User */, the coordinate of the user is at:
please form as: user_location(X,Y).
(X and Y are longitude and latitude)", LOT:"search with 'The geographical coordinates of ... is'").


langda(LLM:"According to the police: /* Police */, the coordinate of the special zone is at:
please form as: special_zone_location(X,Y).
(X and Y are longitude and latitude)").

relative_offset(NorthOffset, EastOffset) :-
langda(LLM:"The location of the danger zone relative to the user, in meters, calculated based on user_location and special_zone_location above
The conversion of longitude should take into account the influence of latitude").

query(relative_offset(NorthOffset, EastOffset)).
query(user_location(North, East)).
"""
model_name = "deepseek-chat"
import re
from langda import langda_solve
from problog import get_evaluatable
special_model = langda_solve("double_dc",danger_rules_string2, model_name, 
            prefix="telegram_bot_special", langda_ext={"User":"Ich bin am Innenstadtcampus der Technischen Universität Darmstadt.", "Police":"Today, the police found a bomb at (49.873094, 8.658995) coordinates. Please stay at least 50 meters away."})
special_result = get_evaluatable().create_from(special_model).evaluate()
special_key = next(iter(special_result))   # 取出唯一 key
pattern = r"relative_offset\(\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)\s*\)"
match = re.search(pattern, str(special_key))

# 49.875,8.65639, 49.875,8.65639
spec_x, spec_y = match.groups()

### =========== processing init_node =========== ###
[{'LOT': "search with 'The geographical coordinates of ... is'", 'NET': None, 'LLM': 'According to the user: /* User */, the coordinate of the user is at:\nplease form as: user_location(X,Y).\n(X and Y are longitude and latitude)', 'FUP': 'True', 'HEAD': '', 'HASH': 'CE079308'}, {'LOT': None, 'NET': None, 'LLM': 'According to the police: /* Police */, the coordinate of the special zone is at:\nplease form as: special_zone_location(X,Y).\n(X and Y are longitude and latitude)', 'FUP': 'True', 'HEAD': '', 'HASH': '2DA01CAA'}, {'LOT': None, 'NET': None, 'LLM': 'The location of the danger zone relative to the user, in meters, calculated based on user_location and special_zone_location above\nThe conversion of longitude should take into account the influence of latitude', 'FUP': 'True', 'HEAD': 'relative_offset(NorthOffset, EastOffset) ', 'HASH': '27C37B02'}]
{'CE079308': 'user_location(49.87500, 8.65639).', '2DA01CAA': 'special_zone_locat

KeyboardInterrupt: 

In [84]:
import re

pattern = r"(\w+)\(\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)\s*\)"
result_list = []
for key in special_result:
    match = re.match(pattern, str(key))
    if match:
        predicate, x, y = match.groups()
        result_list.append((x,y))
        print(f"predicate = {predicate}, x = {x}, y = {y}")


predicate = relative_offset, x = -211.93753657979838, y = 186.67530947625767
predicate = user_location, x = 49.875, y = 8.65639


In [51]:
(x1,y1),(x2,y2) = result_list

In [57]:
float(y2)

8.65639

In [ ]:
danger_rules_string = f"""

langda(LLM:"According to the user: kongresszentrum darmstadium, the coordinate of the user is at:
please form as: user_location(X,Y).
(X and Y are longitude and latitude)", LOT:"search with 'The geographical coordinates of ... is'",FUP:"false").

query(user_location(North, East)).
"""
model_name = "deepseek-chat"
import re
from langda import langda_solve
from problog import get_evaluatable
special_model = langda_solve("double_dc",danger_rules_string, model_name, 
            prefix="telegram_bot_dest", langda_ext={"User":"Ich bin am Innenstadtcampus der Technischen Universität Darmstadt.", "Police":"Today, the police found a bomb at (49.873094, 8.658995) coordinates. Please stay at least 50 meters away."})
special_result = get_evaluatable().create_from(special_model).evaluate()
special_key = next(iter(special_result))   # 取出唯一 key
pattern = r"relative_offset\(\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)\s*\)"
match = re.search(pattern, str(special_key))

# 49.875,8.65639, 49.875,8.65639
spec_x, spec_y = match.groups()

### =========== processing init_node =========== ###
[{'LOT': "search with 'The geographical coordinates of ... is'", 'NET': None, 'LLM': 'According to the user: kongresszentrum darmstadium, the coordinate of the user is at:\nplease form as: user_location(X,Y).\n(X and Y are longitude and latitude)', 'FUP': 'false', 'HEAD': '', 'HASH': '551A768D'}]
{'EFCE1CAE': 'relative_offset(NorthOffset, EastOffset) :- user_location(UserLat, UserLon), special_zone_location(ZoneLat, ZoneLon), NorthOffset is (ZoneLat - UserLat) * 111194.93, EastOffset is (ZoneLon - UserLon) * 111194.93.', '2DA01CAA': 'special_zone_location(49.873094, 8.658995).', 'CE079308': 'user_location(49.87500, 8.65639).'}
[{'551A768D': None}]
processing _decide_next_init ...
### =========== ### current round: 0 ### =========== ###
### =========== processing generate_node =========== ###
Executing first chain: Code generation with tools...


> Entering new AgentExecutor chain...

Invoking: `search_tool` with `{'query': 'The geogr

AttributeError: 'NoneType' object has no attribute 'groups'

In [ ]:
danger_rules_string = f"""

langda(LLM:"According to the user: I just updated my flying plan: [(-230,200), (-215,195), (-200,200), (-185,190), (-170,195), (-150,195), (-130,180),(-115,175), (-100,150), (-90,125), (-80,100), (-60,80), (-40,60), (-30,40),(-20,20), (-13,13), (-7,7), (0,0)],
please form as: fly([]).
store all the points inside the list)", LOT:"search with 'The geographical coordinates of ... is'",FUP:"false").

query(fly(X)).
"""
model_name = "deepseek-chat"
import re
import ast

from langda import langda_solve
from problog import get_evaluatable
special_model = langda_solve("double_dc",danger_rules_string, model_name, 
            prefix="telegram_bot_fly", langda_ext={"User":"Ich bin am Innenstadtcampus der Technischen Universität Darmstadt.", "Police":"Today, the police found a bomb at (49.873094, 8.658995) coordinates. Please stay at least 50 meters away."})
special_result = get_evaluatable().create_from(special_model).evaluate()
special_key = next(iter(special_result))   # 取出唯一 key

key = next(iter(special_result))  # 提取唯一键

# 用正则找出中括号部分
match = re.search(r"fly\((\[.*\])\)", str(key))
if match:
    list_str = match.group(1)       # 获取 "[(-230, 200), ..., (0, 0)]"
    coord_list = ast.literal_eval(list_str)  # 安全地解析为 Python 列表
    print(coord_list)
    # 输出示例：[(-230, 200), (-215, 195), ..., (0, 0)]



### =========== processing init_node =========== ###
[{'LOT': "search with 'The geographical coordinates of ... is'", 'NET': None, 'LLM': 'According to the user: I just updated my flying plan: [(-230,200), (-215,195), (-200,200), (-185,190), (-170,195), (-150,195), (-130,180),(-115,175), (-100,150), (-90,125), (-80,100), (-60,80), (-40,60), (-30,40),(-20,20), (-13,13), (-7,7), (0,0)],\nplease form as: fly([]).\nstore all the points inside the list)', 'FUP': 'false', 'HEAD': '', 'HASH': '88B3E9CC'}]
{}
[{'88B3E9CC': None}]
processing _decide_next_init ...
### =========== ### current round: 0 ### =========== ###
### =========== processing generate_node =========== ###
Executing first chain: Code generation with tools...


> Entering new AgentExecutor chain...

Invoking: `search_tool` with `{'query': 'The geographical coordinates of (-230,200) is'}`


 Running search tool...
[{'title': 'Latitude and Longitude Finder on Map Get Coordinates', 'url': 'https://www.latlong.net/', 'content': 'w

AttributeError: 'NoneType' object has no attribute 'groups'

In [ ]:
# success = promis_execution(result, (offset_x, offset_y), (user_x, user_y), city_attr="darmstadt",load_uam=False)


[(-230, 200), (-215, 195), (-200, 200), (-185, 190), (-170, 195), (-150, 195), (-130, 180), (-115, 175), (-100, 150), (-90, 125), (-80, 100), (-60, 80), (-40, 60), (-30, 40), (-20, 20), (-13, 13), (-7, 7), (0, 0)]


In [31]:
from langda import _problog_test

model111 ="% Your model here\n% Earth constant\nmeter_per_degree(111194.93).\n % R * π / 180 ≈ 111194.93 meters/degree\nuser_location(49.87500, 8.65639).\nspecial_zone_location(49.873094, 8.658995).\nrelative_offset(NorthOffset, EastOffset) :-\n user_location(UserLat, UserLon), special_zone_location(ZoneLat, ZoneLon), NorthOffset is (ZoneLat - UserLat) * 111194.93, EastOffset is (ZoneLon - UserLon) * 111194.93 * cos(UserLat * pi / 180).\nquery(relative_offset(NorthOffset, EastOffset)).\nquery(user_location(North, East))."
model222 ="""
% Your model here
% Earth constant
meter_per_degree(111194.93).
 % R * π / 180 ≈ 111194.93 meters/degree
user_location(49.87500, 8.65639).
special_zone_location(49.873094, 8.658995).
relative_offset(NorthOffset, EastOffset) :-
 user_location(UserLat, UserLon), special_zone_location(ZoneLat, ZoneLon), NorthOffset is (ZoneLat - UserLat) * 111194.93, EastOffset is (ZoneLon - UserLon) * 111194.93 * cos(UserLat * pi / 180).
query(relative_offset(NorthOffset, EastOffset)).
query(user_location(North, East)).
"""

# result = _problog_test(model111)
result = get_evaluatable().create_from(model111).evaluate()

In [4]:
from problog import get_evaluatable
from problog.engine import DefaultEngine
from problog.program import PrologString
from problog.logic import Term
import re
special_model = """
 % Earth constant: R * π / 180 -> 111194.93 meters/degree
user_location(49.87500, 8.65639).
special_zone_location(49.873094, 8.658995).

relative_offset(NorthOffset, EastOffset) :-
    user_location(UserLat, UserLon),
    special_zone_location(ZoneLat, ZoneLon),
    NorthOffset is (ZoneLat - UserLat) * 111194.93,
    EastOffset is (ZoneLon - UserLon) * 111194.93 * cos(UserLat * pi / 180).

query(user_location(X,Y)).
query(relative_offset(X,Y)).
"""

special_model = special_model.strip("'")
special_result = get_evaluatable().create_from(special_model).evaluate()
pattern = r"(\w+)\(\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)\s*\)"
result_list = []
print(special_result)
for key in special_result:
    match = re.match(pattern, str(key))
    if match:
        predicate, x, y = match.groups()
        result_list.append((float(x),float(y)))
        print(f"predicate = {predicate}, x = {x}, y = {y}")
print(result_list)

{user_location(49.875,8.65639): 1.0, relative_offset(-211.93753657979838,186.67530947625767): 1.0}
predicate = user_location, x = 49.875, y = 8.65639
predicate = relative_offset, x = -211.93753657979838, y = 186.67530947625767
[(49.875, 8.65639), (-211.93753657979838, 186.67530947625767)]


In [35]:
result_list

[]